# Stayin' Alive
### *An AI-Powered Tool for Optimal Restaurant and Bar Location Selection and Business Longevity*
42578 – Advanced Business Analytics, DTU, 2025 <br>
Group 21 - Crocs Validation<br>
Giulia Andreatta -sxxxx<br>
Gabriel Lanaro - s233541<br>
Alessia Saccardo - s212246<br>
Gabriele Turetta - sxxxx<br>

### Objective
Opening a restaurant or a bar is a high-risk endeavor—many establishments close within their first few years. In Copenhagen, aspiring restaurateurs and investors often lack a data-driven approach when selecting a location. Moreover, understanding the reasons behind a restaurant’s success or failure remains a challenge.

This project aims to:

- Recommend optimal locations for new restaurants or bars using Survival Analysis.
- Visualize location suitability through an interactive heatmap enriched with predictive longevity scores, pedestrian peak hours, density of restaurants, pins of active and closed activities.

### Datasets

- **Company data scraped from the official CVR registry via [virk.dk](https://datacvr.virk.dk/soegeresultater?fritekst=d&sideIndex=0&size=10)**<br>
Includes business registration details, location, restaurant closures, branchekode.

- **Google Maps Scraped Data**<br>
Includes business location, rating, number of reviews, price range, tags.

- **Pedestrian Dataset from [OpenData.dk](https://www.opendata.dk/city-of-copenhagen/taelling_fodg#:~:text=Number%20of%20pedestrians%20counted%20on,19%20in%20both%20directions)**<br>
Provides foot traffic counts recorded at specific times and locations in Copenhagen.

- **People per Postal Code and Neighbordhood Dataset [Statbank.dk](https://www.statbank.dk/20021)**<br>
Provides the Population at the 1st of January 2025 by municipality, postal code, sex and age.

- **Area per postal code In Copenhagen [Postnumre-da.cybo](https://postnumre-da.cybo.com/danmark/k%C3%B8benhavn/)<br>**
Contains a lot of information about the postal codes but we took only the area of each one.

### ABA Topics Covered
- **Web Data Mining**
Scraping large-scale data from Google Maps and government databases to construct the datasets.

- **Survival Analysis**
Predicting restaurant longevity using Kaplan-Meier and Cox Proportional Hazards models.

- **Recommender Systems**
Suggesting location options for new restaurants and bars based on market gaps and existing competition.

- **AI in the Real World**
Delivering real value to stakeholders by supporting data-driven restaurant planning and resilience strategies.



## 1st Step - Data Scraping from the official CVR registry

This script performs web scraping on the Danish company registry website (https://datacvr.virk.dk)
to extract company details for active business units in specified industry sectors (branchekoder).
It uses Selenium to navigate the search results, extract key information for each business unit,
and follow links to detailed company pages to obtain start and end dates.

The results are saved in a CSV file, and duplicate entries (based on P-number) are avoided by 
keeping track of already seen values. The script is designed to be resumed without duplicating 
previous entries.

Required: chromedriver installed and path correctly set.

Note: The output of the 1st and 2nd cells are already present in the data folder. If you want to run the scraping cells, take in mind that the run takes a very long time!


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time
import csv
import os

# === CONFIGURATION ===
driver_path = r"chromedriver/chromedriver.exe"
options = Options()
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=Service(driver_path), options=options)


# the loop can be run for each branchekode separately, or all at once by uncommenting the lines below.
# In our project, the branchekodes were manually uncommented, to create a single csv file for each branchekode.
# This was done to avoid the script from running for too long, risking on IP bans or website crashes.
# The single csv files were then merged into a single csv file.
branchekodes = [
    # 561110,   # serving food in restaurants and cafes
    # 561190,   # includes the operation of restaurants, where the main emphasis is on takeaway with very limited table service.
    # 563010,   # includes serving beverages, possibly with some edibles, but where the main emphasis is on serving non-alcoholic beverages for immediate consumption on site.
    563020,  #  includes serving beverages, possibly with some edibles, but where the main emphasis is on serving alcoholic beverages for immediate consumption on the premises.
]

# Scraping Structure overview:
# - Main search results are loaded via URL with parameters: sideIndex (pagination), branchekode (industry), etc.
# - Each company entry is a 'div.row' within a 'div[data-cy="soegeresultater-tabel"]'.
# - Basic info (name, address, P-nummer, status, company type) is extracted directly from the search results.
# - For each company, the script follows the "Show More" link in a new browser tab to extract Start date (Startdato)
#   and End date (Ophørsdato), which appear in divs following label tags (either <strong> or <span>).
for branchekode in branchekodes:
    page = 0
    csv_file_path = f"data/scraped_companies_{branchekode}.csv"
    header = [
        "Name",
        "Address",
        "P-nummer",
        "Status",
        "Company Type",
        "Startdate",
        "Enddate",
    ]
    pnummer_seen = set()

    # Load existing P-numbers if the file already exists to avoid duplicates
    file_exists = os.path.exists(csv_file_path)
    if file_exists:
        with open(csv_file_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                pnummer_seen.add(row["P-nummer"])
    else:
        # Create file and write header
        with open(csv_file_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=header)
            writer.writeheader()

    while True:
        url = f"https://datacvr.virk.dk/soegeresultater?sideIndex={page}&enhedstype=produktionsenhed&region=29190623&branchekode={branchekode}"
        print(f"Scraping page {page}")
        driver.get(url)
        time.sleep(2)

        # Get all company rows from the result table
        rows = driver.find_elements(
            By.CSS_SELECTOR, 'div[data-cy="soegeresultater-tabel"] > div.row'
        )

        if not rows:
            print("No data found. Stopping.")
            break

        # Process each company in the current page
        for row in rows:
            try:
                name = row.find_element(By.CSS_SELECTOR, "span.bold.value").text.strip()

                address_block = row.find_element(By.CSS_SELECTOR, "div.col-12.col-lg-4")
                address_lines = address_block.text.strip().split("\n")[-2:]
                address = ", ".join(address_lines)

                pnummer = row.find_element(
                    By.XPATH, './/div[div[text()="P-nummer:"]]/div[2]'
                ).text.strip()

                # Skip if already saved
                if pnummer in pnummer_seen:
                    continue
                pnummer_seen.add(pnummer)

                status = row.find_element(
                    By.XPATH, './/div[div[text()="Status:"]]/div[2]'
                ).text.strip()
                form = row.find_element(
                    By.XPATH, './/div[div[text()="Virksomhedsform:"]]/div[2]'
                ).text.strip()

                link_elem = row.find_element(
                    By.CSS_SELECTOR, 'div[data-cy="vis-mere"] a'
                )
                link = link_elem.get_attribute("href")

                # Open detail page in new tab
                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[1])
                driver.get(link)
                time.sleep(3)

                # Extract dates
                startdato = ""
                ophoersdato = ""

                # Extract start and end dates from the detail page
                try:
                    startdato_element = driver.find_element(
                        By.XPATH,
                        '//div[(strong[text()="Startdato"] or span[text()="Startdato"])]/following-sibling::div',
                    )
                    startdato = startdato_element.text.strip()
                except:
                    startdato = ""

                try:
                    ophoersdato_element = driver.find_element(
                        By.XPATH,
                        '//div[(strong[text()="Ophørsdato"] or span[text()="Ophørsdato"])]/following-sibling::div',
                    )
                    ophoersdato = ophoersdato_element.text.strip()
                except:
                    ophoersdato = ""

                # Close the detail tab and return to the main results tab
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

                # Write to CSV
                with open(csv_file_path, "a", newline="", encoding="utf-8") as f:
                    writer = csv.DictWriter(f, fieldnames=header)
                    writer.writerow(
                        {
                            "Name": name,
                            "Address": address,
                            "P-nummer": pnummer,
                            "Status": status,
                            "Company Type": form,
                            "Startdate": startdato,
                            "Enddate": ophoersdato,
                        }
                    )

                print(f"{name} | {startdato} → {ophoersdato}")

            except Exception as e:
                print("Error during parsing:", e)
                continue

        page += 1
        time.sleep(1)

driver.quit()
print("Scraping finished.")

## 2nd Step - Geocoding Restaurant Addresses Using OpenStreetMap API

This script performs address geocoding using the OpenStreetMap (OSM) API via the geopy library. It takes as input a CSV file containing restaurant records with address fields but missing geographic coordinates (data/scraped_companies_combined_clean.csv). For each address, it attempts to retrieve the corresponding latitude and longitude, which are then saved in a new CSV file.

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import os

# === CONFIGURATION ===
input_file = "data/scraped_companies_combined_clean.csv"
output_file = "data/scraped_companies_combined_clean_with_coords.csv"

# === LOAD ORIGINAL DATA ===
df = pd.read_csv(input_file)


# Function to simplify the address before geocoding
def simplify_address(row):
    addr = str(row["Address"])
    addr = addr.split(",")[0].strip()  # only keep the part before the first comma
    return f"{addr}, Denmark"


# Add coordinate columns if they don't exist
if "latitude" not in df.columns:
    df["latitude"] = None
if "longitude" not in df.columns:
    df["longitude"] = None

# Load already geocoded addresses to avoid duplicates
already_done = set()
if os.path.exists(output_file):
    df_existing = pd.read_csv(output_file)
    already_done = set(df_existing["Address"].dropna().unique())
    print(f"Resuming from {len(already_done)} already completed addresses.")

# Filter rows that still need geocoding
df_to_process = df[~df["Address"].isin(already_done)].copy()
print(f"Addresses to geocode: {len(df_to_process)}")

# Initialize OpenStreetMap geocoder with delay to respect rate limits
geolocator = Nominatim(user_agent="stayin_alive_simple_geocoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5)

# Progressive saving to CSV (append mode!!)
with open(output_file, "a", encoding="utf-8", newline="") as f_out:
    header_written = os.stat(output_file).st_size == 0
    for i, row in df_to_process.iterrows():
        full_address = simplify_address(row)
        try:
            location = geocode(full_address)
            if location:
                row["latitude"] = location.latitude
                row["longitude"] = location.longitude
                print(f"{full_address} -> ({location.latitude}, {location.longitude})")
            else:
                print(f"{full_address} -> not found")
        except Exception as e:
            print(f"Error on {full_address}: {e}")
            continue

        # Append row to output CSV
        pd.DataFrame([row]).to_csv(f_out, index=False, header=header_written)
        header_written = False

## 3rd Step - Scraping Restaurant Metadata from Google Maps with Selenium



This script performs web scraping from Google Maps using Selenium to enrich the dataset of restaurants obtained from steps 1-2 (data/scraped_companies_combined_clean_with_coords.csv). For each restaurant entry (name and address), the script:

1. Opens a Google Maps search page

2. Extracts the official listing title, star rating, number of reviews, price level, and associated category tags if present

3. Saves the collected data into a CSV file

4. The script supports resuming interrupted sessions by skipping entries that have already been saved to the output CSV file.

Required: chromedriver installed and path correctly set.

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import csv
import os

# === CONFIGURATION ===
driver_path = r"chromedriver/chromedriver.exe"
options = Options()
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless")  # Uncomment to run without opening browser window

driver = webdriver.Chrome(service=Service(driver_path), options=options)

# === INPUT & OUTPUT PATHS ===
csv_input_path = r"data/scraped_companies_combined_clean_with_coords.csv"
csv_output_path = r"data/maps_data_scraped.csv"

# Load input data
df_input = pd.read_csv(csv_input_path)
restaurant_data = df_input.to_dict(orient="records")

# Load already saved entries (if output file exists)
saved_entries = set()
header = [
    "Input Name",
    "Input Address",
    "Title",
    "Rating",
    "Reviews",
    "Price Level",
    "Tags",
]

if os.path.exists(csv_output_path):
    with open(csv_output_path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            key = (row["Input Name"], row["Input Address"])
            saved_entries.add(key)
else:
    with open(csv_output_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()

# === MAIN LOOP OVER RESTAURANTS ===
for entry in restaurant_data:
    name = entry["Name"]
    address = entry["Address"]
    key = (name, address)

    if key in saved_entries:
        continue

    try:
        print(f"Searching: {name} @ {address}")
        query = f"{name} {address}".replace(" ", "+")
        linkmaps = f"https://www.google.com/maps/search/{query}"
        print(f"URL: {linkmaps}")
        driver.get(linkmaps)
        time.sleep(2)

        try:
            title = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf").text
        except:
            title = ""

        try:
            rating = driver.find_element(
                By.CSS_SELECTOR, 'div.F7nice > span span[aria-hidden="true"]'
            ).text
        except:
            rating = ""

        try:
            reviews_elem = driver.find_element(
                By.CSS_SELECTOR, 'div.F7nice > span span[aria-label$="reviews"]'
            ).text
            reviews = reviews_elem.strip("()")
        except:
            reviews = ""

        try:
            price_level = driver.find_element(
                By.CSS_SELECTOR, "div.DfOCNb.fontBodyMedium > div"
            ).text.split("\n")[0]
        except:
            price_level = ""

        try:
            outer_divs = driver.find_elements(By.CSS_SELECTOR, "div.KNfEk.aUjao")
            tags = []
            for div in outer_divs:
                try:
                    tag = div.find_element(
                        By.CSS_SELECTOR, "div.tXNTee span.uEubGf.fontBodyMedium"
                    ).text
                    tags.append(tag)
                except:
                    continue
            tags = ", ".join(tags)
        except:
            tags = ""

        # Append to CSV immediately
        with open(csv_output_path, "a", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=header)
            writer.writerow(
                {
                    "Input Name": name,
                    "Input Address": address,
                    "Title": title,
                    "Rating": rating,
                    "Reviews": reviews,
                    "Price Level": price_level,
                    "Tags": tags,
                }
            )

        print(f"Saved: {title}")

    except Exception as e:
        print("Error during scraping:", e)
        continue

driver.quit()
print("Scraping finished.")

## 4th Step – Cleaning and Preparing the Population by Postal Code Dataset


The original dataset included the following fields: **ID**, **Neighborhood**, **Total People**, **Total Men**, and **Total Women**. Preliminary data cleaning was performed primarily in Excel for convenience. The key steps were:

1. The **Neighborhood** column was split into two separate fields: **Postal Code** and **Neighborhood Name**, resulting in the file `cph_population_clean.csv`.
2. This cleaned dataset was merged with a postal code area reference table to enable the calculation of population density per postal code.
3. Population and restaurant densities were calculated, producing the final dataset `population_df.csv`.

In [11]:
import pandas as pd
import glob
import os
import re

# Step 1: Load cleaned population dataset
population_df = pd.read_csv(
    "data/cph_population_clean.csv",
    encoding="ISO-8859-1",
)

# Step 2: Collect all company data CSVs from the specified folder
# These files follow a naming convention like scraped_companies_<code>_active.csv
file_paths = glob.glob("data/Temp/scraped_companies_*.csv")

all_dfs = []

# Step 3: Loop through each company CSV file
for path in file_paths:
    filename = os.path.basename(path)

    # Extract the company identifier code from the filename
    try:
        code = filename.split("_")[2]
    except IndexError:
        code = "000000"  # Use fallback code if filename is not in expected format

    # Determine if the file refers to active companies
    is_active = filename.lower().endswith("active.csv")

    # Load the current CSV file
    df = pd.read_csv(path)

    # Add identifier and status columns to the DataFrame
    df["code"] = code
    df["active"] = is_active

    # Store in the list for later concatenation
    all_dfs.append(df)

# Step 4: Combine all company DataFrames into one
merged_df = pd.concat(all_dfs, ignore_index=True)

# Optional: Save merged file (currently commented out)
# merged_df.to_csv("merged_companies.csv", index=False)

print("Merge completed. File saved as merged_companies.csv")


# Step 5: Extract the postal code from the Address column using regex
def extract_postal_code(address):
    try:
        # Look for a pattern like ", 2100" where 2100 is the postal code
        match = re.search(r",\s*(\d{4})", address)
        return match.group(1) if match else None
    except:
        return None


# Apply the postal code extraction function
merged_df["postal_code"] = merged_df["Address"].apply(extract_postal_code)

# Drop columns that are not needed for the analysis
merged_df = merged_df.drop(columns=["Status"])
merged_df = merged_df.drop(columns=["Company Type"])

# Standardize column names and formats for merging
population_df = population_df.rename(columns={"neighborhood_id": "neighborhood_code"})
population_df = population_df.replace("-", 0)  # Replace dashes with 0 (if any)

# Ensure postal codes are strings to prevent merge issues
population_df["postal_code"] = population_df["postal_code"].astype(str)
population_df["postal_code_m2"] = population_df["postal_code_m2"].astype(int)
merged_df["postal_code"] = merged_df["postal_code"].astype(str)

# Step 6: Compute population density (people per km²)
population_df["population_density_km2"] = population_df["Total"] / (
    population_df["postal_code_m2"] / 1_000_000
)

# Step 7: Count how many restaurants (companies) exist per postal code
restaurant_counts = (
    merged_df.groupby("postal_code").size().reset_index(name="restaurant_count")
)

# Step 8: Merge restaurant counts into the population DataFrame
population_df = population_df.merge(restaurant_counts, on="postal_code", how="left")

# Fill any missing values (postal codes with 0 restaurants) with 0
population_df["restaurant_count"] = (
    population_df["restaurant_count"].fillna(0).astype(int)
)

# Step 9: Save final dataset with population and restaurant density info
population_df.to_csv("data/population_df.csv", index=False)

# Print a sample of the final DataFrame
print(population_df.head())

Merge completed. File saved as merged_companies.csv
   neighborhood_code neighborhood_name postal_code  postal_area  \
0                101         København        1050  København K   
1                101         København        1051  København K   
2                101         København        1052  København K   
3                101         København        1053  København K   
4                101         København        1054  København K   

   postal_code_m2  Total  Men  Women  population_density_km2  restaurant_count  
0           39053     20   10     10              512.124549                16  
1           55643    399  191    208             7170.713297                28  
2            6563    421  204    217            64147.493524                 2  
3            2434    404  181    223           165981.922761                 2  
4            4953    392  201    191            79143.953160                 1  


## 5th Step - 1st Heatmap, an interactive Spatial Visualization of Restaurants/Bars and Pedestrian Traffic in Copenhagen

This script generates an interactive Folium heatmap that visualizes restaurant/bar locations and pedestrian traffic in Copenhagen. It combines multiple layers of spatial data to support exploratory analysis for business location decisions. 
This initial heatmap with multiple layers serves as a visual foundation for later overlaying the survival analysis scores as a new layer in the heatmap. By combining all the layers, it will be possible fx to compare areas of high restaurant density and high pedestrian density with predicted survival outcomes, helping to identify not only where restaurants are concentrated, but also where they are most likely to succeed over time.

This first heatmap was obtained by combining the restaurants/bars dataset obtained in the first 2 steps (data/scraped_companies_combined_clean_with_coords.csv) and the Pedestrian Dataset downloaed from OpenData.dk (data/foot_traffic.csv)

Code key functionalities:
- Restaurant Heatmap: Shows the density of restaurant locations.
- Longevity Heatmap: Visualizes how long restaurants have stayed open, based on registration and closure dates.
- Status Markers: Differentiates between currently active and closed restaurants with green and red markers.
- Branchekode Filter: Allows filtering restaurants by industry classification code (branchekode).
- Pedestrian Traffic Heatmap: Displays average daily foot traffic (7 AM–7 PM) from official measurements.
- Peak Hour Traffic Circles: Highlights high-density areas during peak foot traffic (7 AM–7 PM) using proportional red circles.

The result is an interactive map saved as an HTML file, enabling users to toggle layers, explore patterns, and identify high-potential zones for business development.

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# Load restaurant dataset with coordinates
restaurants_df = pd.read_csv("data/scraped_companies_combined_clean_with_coords.csv")
restaurants_df = restaurants_df.dropna(subset=["latitude", "longitude"])

# Load pedestrian traffic dataset with coordinates
traffic_df = pd.read_csv("data/foot_trafic.csv")
traffic_df = traffic_df.dropna(subset=["lat", "lon"])

# Preprocessing longevity
restaurants_df["startdate"] = pd.to_datetime(
    restaurants_df["startdate"], errors="coerce"
)
restaurants_df["enddate"] = pd.to_datetime(restaurants_df["enddate"], errors="coerce")
restaurants_df["enddate_filled"] = restaurants_df["enddate"].fillna(
    pd.Timestamp.today()
)
restaurants_df["longevity_days"] = (
    restaurants_df["enddate_filled"] - restaurants_df["startdate"]
).dt.days

# Initialize the map centered on Copenhagen
map_ = folium.Map(location=[55.6761, 12.5683], zoom_start=13)

# --- HEATMAP: Restaurants ---
heat_points = restaurants_df[["latitude", "longitude"]].values.tolist()
heatmap_layer = folium.FeatureGroup(name="Restaurants Heatmap")
HeatMap(heat_points, radius=10, blur=15).add_to(heatmap_layer)
heatmap_layer.add_to(map_)

# --- HEATMAP: Longevity ---
longevity_points = (
    restaurants_df[["latitude", "longitude", "longevity_days"]].dropna().values.tolist()
)
longevity_layer = folium.FeatureGroup(name="Restaurants Longevity Heatmap", show=False)
HeatMap(longevity_points, radius=15, blur=25, max_zoom=14).add_to(longevity_layer)
longevity_layer.add_to(map_)

# --- MARKERS: Active / Closed Restaurants ---
active_layer = folium.FeatureGroup(name="Active Restaurants", show=False)
closed_layer = folium.FeatureGroup(name="Closed Restaurants", show=False)

for _, row in restaurants_df.iterrows():
    popup = folium.Popup(
        f"<b>{row.get('name', 'N/A')}</b><br>"
        f"Business Code: {row.get('branchekode', 'N/A')}<br>"
        f"Status: {row.get('status', 'N/A')}<br>"
        f"Opening Date: {row.get('startdate', 'N/A')}<br>"
        f"Postal Code: {row.get('zip', 'N/A')}",
        max_width=300,
    )
    marker = folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=popup,
        icon=folium.Icon(color="green" if row.get("active", False) else "red"),
    )
    if row.get("active", False):
        marker.add_to(active_layer)
    else:
        marker.add_to(closed_layer)

active_layer.add_to(map_)
closed_layer.add_to(map_)

# --- FILTER: Branchekode ---
branche_layer_dict = {}
for branche in restaurants_df["branchekode"].dropna().unique():
    layer = folium.FeatureGroup(name=f"Branchekode: {branche}", show=False)
    for _, row in restaurants_df[restaurants_df["branchekode"] == branche].iterrows():
        popup = folium.Popup(
            f"<b>{row.get('name', 'N/A')}</b><br>"
            f"Branchekode: {row.get('branchekode', 'N/A')}<br>"
            f"Status: {row.get('status', 'N/A')}<br>"
            f"Startdate: {row.get('startdate', 'N/A')}<br>"
            f"ZIP: {row.get('zip', 'N/A')}",
            max_width=300,
        )
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=4,
            color="blue",
            fill=True,
            fill_opacity=0.6,
            popup=popup,
        ).add_to(layer)
    layer.add_to(map_)

# --- PEDESTRIAN TRAFFIC: HEATMAP aadt_fod_7_19 ---
heat_traffic_points = (
    traffic_df[["lat", "lon", "aadt_fod_7_19"]].dropna().values.tolist()
)
heatmap_ped_layer = folium.FeatureGroup(name="Pedestrian Heatmap (7-19)")
HeatMap(heat_traffic_points, radius=15, blur=25, max_zoom=14).add_to(heatmap_ped_layer)
heatmap_ped_layer.add_to(map_)

# --- PEDESTRIAN TRAFFIC: CIRCLE LAYER hvdt_fod_7_19 ---
circle_layer = folium.FeatureGroup(name="Pedestrian Peak Hour 7-19", show=False)
for _, row in traffic_df.iterrows():
    value = row.get("hvdt_fod_7_19")
    if pd.notna(value):
        radius = value / 500  # scaling factor
        popup = folium.Popup(
            f"<b>{row.get('vejnavn', '')}</b><br>"
            f"Peak Hour 7-19: {int(value)}<br>"
            f"Description: {row.get('beskrivelse', '')}<br>"
            f"Date: {row.get('taelle_dato', '')}",
            max_width=300,
        )
        folium.CircleMarker(
            location=[row["lat"], row["lon"]],
            radius=radius,
            color="red",
            fill=True,
            fill_opacity=0.5,
            popup=popup,
        ).add_to(circle_layer)
circle_layer.add_to(map_)

# Add layer control to enable toggling layers
folium.LayerControl(collapsed=False).add_to(map_)

# Save the final map
map_.save("output/interactive_map_with_filtered_traffic.html")

The final aim of this project is to add a new layer to the existing heatmap that visualizes the results of the survival analysis. To perform the analysis and produce an output that could be displayed as a heatmap, it has been created a dataset in which each row corresponds to a unique geographic coordinate. The following sections describe the construction of the dataset used for the survival analysis.

## 6th Step - Expanding the Pedestrian Traffic Dataset to the entire Copenhagen municipality 

This script expands the pedestrian traffic dataset to cover the entire Copenhagen municipality. It first generates a 200 m regular grid over the commune and then it clips the points to the official municipal boundary using its GeoJSON shape. To each grid point is assigned average daily and peak pedestrian counts via inverse-distance weighting (IDW), which computes a weighted average of nearby measurements with weights proportional to \(1/d^3\). The result is a dataset of over 7 000 points, each spaced 200 m apart and holding a value of average daily foot traffic (7 AM–7 PM) and peak foot traffic (7 AM–7 PM).

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import unary_union

# Load and clean pedestrian counts
ped = pd.read_csv("data/foot_trafic.csv", parse_dates=["taelle_dato"])
ped["longitude"] = ped["wkb_geometry"].str.extract(r"POINT \(([^ ]+)")[0].astype(float)
ped["latitude"] = (
    ped["wkb_geometry"].str.extract(r"POINT \([^ ]+ ([^ ]+)\)")[0].astype(float)
)
ped = ped.dropna(subset=["latitude", "longitude"])
ped["aadt"] = pd.to_numeric(ped["aadt_fod_7_19"], errors="coerce")
ped["hvdt"] = pd.to_numeric(ped["hvdt_fod_7_19"], errors="coerce")
ped = ped.dropna(subset=["aadt", "hvdt"])

# Project to metric CRS for distance calculations
ped_gdf = gpd.GeoDataFrame(
    ped[["aadt", "hvdt", "longitude", "latitude"]],
    geometry=gpd.points_from_xy(ped.longitude, ped.latitude),
    crs="EPSG:4326",
).to_crs(epsg=3857)

print(f"Original pedestrian points: {len(ped_gdf)}")

# Build 200 m grid over Copenhagen
munis = gpd.read_file(
    "https://raw.githubusercontent.com/"
    "magnuslarsen/geoJSON-Danish-municipalities/"
    "master/municipalities/municipalities.geojson"
).to_crs(epsg=3857)
cph_union = munis[munis["label_dk"] == "København"].unary_union
spacing = 200
minx, miny, maxx, maxy = cph_union.bounds
xs = np.arange(minx, maxx + spacing, spacing)
ys = np.arange(miny, maxy + spacing, spacing)

grid = (
    gpd.GeoDataFrame(geometry=[Point(x, y) for x in xs for y in ys], crs="EPSG:3857")
    .loc[lambda df: df.within(cph_union)]
    .reset_index(drop=True)
)

print(f"Grid points after clipping: {len(grid)}")


# IDW interpolation function
def idw(point, xs, ys, values, power=3):
    d = np.hypot(xs - point.x, ys - point.y)
    w = 1.0 / (d**power + 1e-6)
    return (w * values).sum() / w.sum()


# Prepare arrays for interpolation
xs_pts = ped_gdf.geometry.x.values
ys_pts = ped_gdf.geometry.y.values
aadt_vals = ped_gdf["aadt"].values
hvdt_vals = ped_gdf["hvdt"].values

# Compute interpolated traffic counts
grid["aadt_fod_7_19"] = grid.geometry.apply(lambda p: idw(p, xs_pts, ys_pts, aadt_vals))
grid["hvdt_fod_7_19"] = grid.geometry.apply(lambda p: idw(p, xs_pts, ys_pts, hvdt_vals))

# Reproject to lat/lon and save
out = grid.to_crs(epsg=4326)
out["latitude"] = out.geometry.y
out["longitude"] = out.geometry.x
out[["latitude", "longitude", "aadt_fod_7_19", "hvdt_fod_7_19"]].to_csv(
    "data/expanded_foot_trafic.csv", index=False
)

print("Saved interpolated grid.")

Original pedestrian points: 403


/tmp/ipykernel_12283/747123367.py:37: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  cph_union = munis[munis["label_dk"] == "København"].unary_union


Grid points after clipping: 7062
Saved interpolated grid.


## 7th Step - Merge Pedestrian Traffic with Postal Area Demographics

This script takes three inputs:
- the DAWA GeoJSON of Danish postal-code areas, containing the polygon boundaries for each `postal_code`  
- a CSV of population statistics by postal code, containing `neighborhood_code`, `neighborhood_name`, `postal_code`, `postal_area`, `postal_code_m2`, `Total`, `Men`, `Women`, `population_density_km2`
- the interpolated pedestrian traffic CSV from the previous step, containing `latitude`, `longitude`, `aadt_fod_7_19`, and `hvdt_fod_7_19`  

It combines them into one clean output. First, it computes the geometric centroid of each postal-code polygon and merges these centroids with the population figures by matching on `postal_code`. Next for each point it checks which postal polygon contains it. Any points that fall outside all postal areas or lack full population data are removed. The result is a single CSV where each row gives a point’s latitude, longitude, interpolated pedestrian counts, its `postal_code`, and the linked demographic attributes.  


In [ ]:
import geopandas as gpd
import pandas as pd

# Load postal areas and compute centroids
postal_gdf = gpd.read_file("https://dawa.aws.dk/postnumre?format=geojson").to_crs(
    epsg=4326
)
postal_gdf["zip_centroid"] = postal_gdf.geometry.centroid
postal_gdf["zip_lon"] = postal_gdf.zip_centroid.x
postal_gdf["zip_lat"] = postal_gdf.zip_centroid.y
postal_gdf = postal_gdf.rename(columns={"nr": "postal_code"})[
    ["postal_code", "zip_lat", "zip_lon", "geometry"]
]

# Load population data and merge
pop = pd.read_csv("data/population_df.csv", dtype={"postal_code": str})
pop["postal_code"] = pop["postal_code"].str.zfill(4)
postal_with_pop = postal_gdf.merge(pop, on="postal_code", how="left")

# Load interpolated pedestrian traffic and convert to GeoDataFrame
ped = pd.read_csv("data/expanded_foot_trafic.csv")
ped_gdf = gpd.GeoDataFrame(
    ped, geometry=gpd.points_from_xy(ped.longitude, ped.latitude), crs="EPSG:4326"
)

# Spatial join pedestrians to postal areas
joined = gpd.sjoin(ped_gdf, postal_with_pop, how="left", predicate="within")

# Remove rows where population stats are missing
joined = joined.dropna(subset=["Total", "Men", "Women", "population_density_km2"])

# Select final columns and save
joined[
    [
        "latitude",
        "longitude",
        "aadt_fod_7_19",
        "hvdt_fod_7_19",
        "postal_code",
        "Total",
        "Men",
        "Women",
        "population_density_km2",
    ]
].to_csv("data/pedestrian_with_zip_stats.csv", index=False)

/tmp/ipykernel_12283/422835725.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  postal_gdf["zip_centroid"] = postal_gdf.geometry.centroid


## 8th Step - Creating the Final Dataset for Survival Analysis by adding restaurant information

This script creates the final dataset used to perform the survival analysis. To build this dataset, two sources are merged:

- **`pedestrian_with_zip_stats.csv`**, generated in the previous step, contains a 200-meter grid covering the Copenhagen municipality. Each point includes interpolated pedestrian traffic values and demographic data such as population density and gender breakdown.
- **`mergedFinal.csv`**, a dataset of restaurants that includes the following fields:
  - `Restaurant_Name`, `Restaurant_Address`: identification and location details  
  - `Opening_Date`, `Closing_Date`: used to calculate how long each restaurant has been in operation  
  - `Branchekod`: an industry code classifying the type of restaurant  
  - `Postal_Code`: used for linking with demographic data  
  - `Rating`, `Number_of_Reviews`, `Price_Level`: customer review and pricing information  
  - `Latitude`, `Longitude`: geographic coordinates of the restaurant  

The goal of the script is to assign to each coordinate in the grid the aggregated characteristics of restaurants located within a 200-meter radius. First, a circular buffer is created around each grid point, and restaurants that fall within these buffers are identified using a spatial join. For each grid point, the script computes several statistics: the total number of restaurants nearby, how many are still open or have closed, their average and median survival times (in days, months, and years), the number of openings and closures in the last three years, and the mean values of customer ratings, number of reviews, and price levels.

Additionally, the frequency of each restaurant type (based on `Branchekod`) is computed and added as separate columns. These aggregated values are merged back into the original grid, which is then reprojected to latitude and longitude.

The result is saved as `df_survival_analysis.csv`.

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime

radius_m = 200

# Load grid with zip/pop and convert to metric CRS
grid = pd.read_csv("data/pedestrian_with_zip_stats.csv")
grid_gdf = gpd.GeoDataFrame(
    grid, geometry=gpd.points_from_xy(grid.longitude, grid.latitude), crs="EPSG:4326"
).to_crs(epsg=3857)

# Load and prepare restaurant data
rest = pd.read_csv(
    "data/mergedFinal_ds.csv",
    parse_dates=["Opening_Date", "Closing_Date"],
    dayfirst=True,
).rename(
    columns={
        "Opening_Date": "startdate",
        "Closing_Date": "enddate",
        "Price_Level": "price_level",
    }
)
rest_gdf = gpd.GeoDataFrame(
    rest, geometry=gpd.points_from_xy(rest.Longitude, rest.Latitude), crs="EPSG:4326"
).to_crs(epsg=3857)

# Compute survival time
today = pd.Timestamp(datetime.today())
rest_gdf["survival_days"] = (
    rest_gdf["enddate"].fillna(today) - rest_gdf["startdate"]
).dt.days

# Buffer grid points and spatially join restaurants
buffers = grid_gdf.copy()
buffers["geometry"] = buffers.geometry.buffer(radius_m)
joined = gpd.sjoin(
    rest_gdf, buffers[["geometry"]], how="inner", predicate="within"
).rename(columns={"index_right": "grid_index"})


# Aggregate restaurant stats per grid cell
def agg_stats(df):
    now3 = today - pd.DateOffset(years=3)
    sd = df["survival_days"]
    return pd.Series(
        {
            "num_restaurants_total": len(df),
            "num_restaurants_open": df["enddate"].isna().sum(),
            "num_restaurants_closed": df["enddate"].notna().sum(),
            "avg_survival_days": sd.mean(),
            "median_survival_days": sd.median(),
            "avg_survival_months": sd.mean() / 30.44,
            "avg_survival_years": sd.mean() / 365.25,
            "openings_last_3_years": (df["startdate"] >= now3).sum(),
            "closures_last_3_years": (df["enddate"] >= now3).sum(),
            "mean_rating": df["Rating"].mean(),
            "mean_reviews": df["Number_of_Reviews"].mean(),
            "mean_price_level": df["price_level"].mean(),
        }
    )


stats = joined.groupby("grid_index").apply(agg_stats)

# Pivot branch codes and join
code_counts = (
    joined.groupby(["grid_index", "Branchekod"])
    .size()
    .unstack(fill_value=0)
    .add_prefix("code_")
)
stats = stats.join(code_counts, how="left").fillna(0)

# Merge back onto grid, reproject, extract coords
result = grid_gdf.join(stats, how="left").fillna(0).to_crs(epsg=4326)
result["latitude"] = result.geometry.y
result["longitude"] = result.geometry.x

# Select final columns and save
base_cols = [
    "latitude",
    "longitude",
    "aadt_fod_7_19",
    "hvdt_fod_7_19",
    "postal_code",
    "Total",
    "Men",
    "Women",
    "population_density_km2",
    "num_restaurants_total",
    "num_restaurants_open",
    "num_restaurants_closed",
    "avg_survival_days",
    "median_survival_days",
    "avg_survival_months",
    "avg_survival_years",
    "openings_last_3_years",
    "closures_last_3_years",
    "mean_rating",
    "mean_reviews",
    "mean_price_level",
]
code_cols = [c for c in stats.columns if c.startswith("code_")]
final_df = result[base_cols + code_cols]

final_df.to_csv("data/df_survival_analysis.csv", index=False)
print("Saved df_survival_analysis.csv")

Saved df_survival_analysis.csv


The table below describes the columns included in the final dataset `df_survival_analysis.csv` where each row represents a 200-meter grid point and includes pedestrian traffic, demographic statistics, and aggregated information about nearby restaurants.  


| Column                     | Description                                                                                       |
|----------------------------|---------------------------------------------------------------------------------------------------|
| **latitude**               | Grid point latitude (decimal degrees, WGS84)                                                      |
| **longitude**              | Grid point longitude (decimal degrees, WGS84)                                                     |
| **aadt_fod_7_19**          | Interpolated average daily pedestrian volume (between 07:00–19:00)                                 |
| **hvdt_fod_7_19**          | Interpolated peak pedestrian volume (between 07:00–19:00)                                          |
| **postal_code**            | Danish postal code containing the grid point                                                      |
| **Total**                  | Total population in the postal area                                                               |
| **Men**                    | Male population in the postal area                                                                |
| **Women**                  | Female population in the postal area                                                              |
| **population_density_km2** | Population density (people per km²) in the postal area                                            |
| **num_restaurants_total**  | Total number of restaurants within 200 m of the grid point                                        |
| **num_restaurants_open**   | Number of those restaurants still open                                                            |
| **num_restaurants_closed** | Number of those restaurants that have closed                                                      |
| **avg_survival_days**      | Mean restaurant “survival” time (days between opening and closing or today)                       |
| **median_survival_days**   | Median restaurant “survival” time (days)                                                          |
| **avg_survival_months**    | Mean survival time converted to months (days/30.44)                                               |
| **avg_survival_years**     | Mean survival time converted to years (days/365.25)                                               |
| **openings_last_3_years**  | Count of restaurant openings in the past 3 years                                                  |
| **closures_last_3_years**  | Count of restaurant closures in the past 3 years                                                  |
| **mean_rating**            | Average customer rating of restaurants in the buffer                                              |
| **mean_reviews**           | Average number of reviews per restaurant in the buffer                                            |
| **mean_price_level**       | Average price-level indicator of restaurants in the buffer                                        |
| **code_<Branchekod>**      | Number of restaurants with branch code `<Branchekod>` within 200 m (one column per branch code)   |
